In [381]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import random
import operator

In [382]:
# width = 400
# height = 400

# block_size = 80
# num_blocks_x = width // block_size
# num_blocks_y = height // block_size

# image = np.ones((height, width), dtype=np.uint8) * 255

# for i in range(num_blocks_y):
#     for j in range(num_blocks_x):
#         block_type = np.random.choice(['white', 'black', 'mixed'], p=[1/3, 1/3, 1/3])
#         if block_type == 'white':
#             image[i*block_size:(i+1)*block_size, j*block_size:(j+1)*block_size] = 255
#         elif block_type == 'black':
#             image[i*block_size:(i+1)*block_size, j*block_size:(j+1)*block_size] = 0
#         elif block_type == 'mixed':
#             image[i*block_size:(i+1)*block_size, j*block_size:(j+1)*block_size] = np.random.choice([0, 255])

# cv.imwrite("random_image.png", image)

# plt.imshow(image, cmap='gray')
# plt.axis('off')
# plt.show()

In [383]:
im_gray = cv.imread('random_image.png', cv.IMREAD_GRAYSCALE)

# plt.imshow(im_gray, cmap='gray')
# plt.axis('off')
# plt.show()
# print("Image shape:", im_gray.shape)

# thresh, im_bw = cv.threshold(im_gray, 128, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)
thresh = 80
im_bw = cv.threshold(im_gray, thresh, 1, cv.THRESH_BINARY)[1]

# plt.imshow(im_bw, cmap='gray')
# plt.axis('off')
# plt.show()
print(im_bw.shape)


(400, 400)


In [384]:
def decimalToBinary(n):
    return bin(n).replace("0b", "")

def binaryTodecimal(n):
    decimal = 0
    power = 1
    while n>0:
        rem = n%10
        n = n//10
        decimal += rem*power
        power = power*2
        
    return decimal

def calc_factors(dimension):
    n_factors = []
    for i in range(1, dimension + 1):
        if dimension % i == 0:
            n_factors.append(i)
    n = random.choice(n_factors)
    return(n)
def check(dimension):
    n_factors = []
    for i in range(1, dimension + 1):
        if dimension % i == 0:
            n_factors.append(i)
    return n_factors

# print(decimalToBinary(8))
class Window:
    def __init__(self,p,q):
        self.p=p
        self.q=q
        self.fitness=0
    def get_size(self):
        print(self.p,self.q)
        
class CAC:
    population=[]
    fittest=Window(0,0)

    def __init__(self,population_size,img):
        self.population_size=population_size
        self.img=img
        self.generate_population()
        
        self.select_parents()
        ctr=0
        while(ctr<10):
            print("counter=",ctr)

            parent_1=self.select_parents()
            parent_2=self.select_parents()

            child_1,child_2 = self.crossover(parent_1,parent_2) 

            self.population.append(child_1)
            self.population.append(child_2)

            
            self.population.sort(key=lambda x:x.fitness,reverse=True)
            print(self.population[0].fitness)
            print(self.fittest.fitness)
            
            if(self.population[0] == self.fittest):
                ctr=ctr+1
            elif(self.population[0] > self.fittest):
                self.fittest = self.population[0]
            else:
                ctr=0
        print("Optimal window size PxQ:",self.fittest.p,"X",self.fittest.q,sep=' ')
        print("Optimal window fitness:",self.fittest.fitness)

    def generate_population(self):
        for _ in range(self.population_size):
            
            p=calc_factors(self.img.shape[0])
            q=calc_factors(self.img.shape[1])
            
            new_window=Window(p, q)
            blocks=self.assign_blocks(new_window)
            self.fitness(new_window,blocks)
            self.population.append(new_window)

    def assign_blocks(self,window):
        p=window.p
        q=window.q
        blocks = {'B':0,'W':0,'M':0}
        for i in range(0, self.img.shape[0] - p + 1, p):
            for j in range(0, self.img.shape[1] - q + 1, q):
                window = im_bw[i:i+p, j:j+q]

        if(np.sum(window)==0):
            blocks['B']=blocks['B']+1
        elif(np.sum(window)==len(window)):
            blocks['W']=blocks['W']+1
        else:
            blocks['M']=blocks['M']+1

        return blocks

    def fitness(self,window,blocks):
        highest_value_key=max(blocks.items(), key=operator.itemgetter(1))[0]
        arr=[]
        for obj in blocks:
            if(obj==highest_value_key and obj):
                arr.append(blocks[obj])
            else:                
                arr.append(blocks[obj]*2)
            if (obj=='M'):
                arr.append(window.p*window.q)

        n1=(self.img.shape[0])*(self.img.shape[1])
        n2=sum(arr)
        fitness_val = n1 / n2
        window.fitness=fitness_val

        if(fitness_val>self.fittest.fitness):
            self.fittest=window
        # print(fitness_val)
        # return fitness_val
    

    def select_parents(self):
        tournament_size = 3
        tournament = random.sample(self.population, tournament_size)
        return max(tournament, key=lambda obj: obj.fitness)


    def crossover(self,parent1, parent2):
        parent_1_binary_p=decimalToBinary(parent1.p)
        parent_1_binary_q=decimalToBinary(parent1.q)

        parent_2_binary_p=decimalToBinary(parent2.p)
        parent_2_binary_q=decimalToBinary(parent2.q)

        parent1=str(parent_1_binary_p)+str(parent_1_binary_q)
        parent2=str(parent_2_binary_p)+str(parent_2_binary_q)

        # parent1=decimalToBinary(parent1)
        # parent2=decimalToBinary(parent2)

        split_point = random.randint(1, len(parent1) - 1)

        child1 = parent1[:split_point] + parent2[split_point:]
        child2 = parent2[:split_point] + parent1[split_point:]

        child1=self.mutation(child1)
        child2=self.mutation(child2)

        child_1_obj=Window(0,0)
        child_2_obj=Window(0,0)

        # child1_decimal = binaryTodecimal(child1)
        # child2_decimal = binaryTodecimal(child2)
        child_1_obj.p=int(child1[:len(child1)], 2)
        child_1_obj.q=int(child1[len(child1)-1:], 2)

        child_2_obj.p=int(child2[:len(child2)], 2)
        child_2_obj.q=int(child2[len(child2)-1:], 2)
        # child1_decimal=int(child1, 2)
        # child2_decimal=int(child2, 2)
        if(child_1_obj.p not in check(self.img.shape[0])):
            child_1_obj.p = calc_factors(self.img.shape[0])
        if(child_1_obj.q not in check(self.img.shape[1])):
            child_1_obj.q = calc_factors(self.img.shape[1])

        if(child_2_obj.p not in check(self.img.shape[0])):
            child_2_obj.p = calc_factors(self.img.shape[0])
        if(child_2_obj.q not in check(self.img.shape[1])):
            child_2_obj.q = calc_factors(self.img.shape[1])

        b_1=self.assign_blocks(child_1_obj)
        self.fitness(child_1_obj,b_1)

        b_2=self.assign_blocks(child_2_obj)
        self.fitness(child_2_obj,b_2)

        return child_1_obj, child_2_obj
    

    def mutation(self,individual):
        chromosome = list(str(individual))
        if random.random() < 0.1:
            index = random.randint(0, len(chromosome) - 1)
            if (chromosome[index]):
                chromosome[index]='0'
            else:
                chromosome[index]='1'
        individual = str(''.join(chromosome))
        return individual   


In [385]:
myObj=CAC(100,im_bw)

window_height, window_width = im_bw.shape

p=calc_factors(window_height)
q=calc_factors(window_width)

# print(window_height,window_width)
# print(p,q)

# for window in myObj.population:
#     print(window.fitness)



counter= 0
80000.0
80000.0
counter= 1
80000.0
80000.0
counter= 2
80000.0
80000.0
counter= 3
80000.0
80000.0
counter= 4
80000.0
80000.0
counter= 5
80000.0
80000.0
counter= 6
80000.0
80000.0
counter= 7
80000.0
80000.0
counter= 8
80000.0
80000.0
counter= 9
80000.0
80000.0
Optimal window size PxQ: 1 X 1
Optimal window fitness: 80000.0
